In [1]:
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import hist
# from hist import Hist
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import hist
from topcoffea.modules.histEFT import HistEFT
NanoAODSchema.warn_missing_crossrefs = False

from coffea.analysis_tools import PackedSelection
from topcoffea.modules import utils
import topcoffea.modules.eft_helper as efth

In [2]:
# histogram style
hep.style.use("CMS")
params = {'axes.labelsize': 20,
          'axes.titlesize': 20,
          'legend.fontsize':20}
plt.rcParams.update(params)

In [3]:
fname = "/cms/cephfs/data/store/user/hnelson2/mc/RunIISummer16NanoAODv6/ST_tW_antitop_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/15F67CF8-4FA2-424C-ADA6-4A1065BDBB1D.root"

In [4]:
# Load in events from root file
events = NanoEventsFactory.from_root(
    fname,
    schemaclass=NanoAODSchema.v6,
    metadata={"dataset": "TLNuto2L2Nu"},
).events()

In [6]:
events.genWeight

<Array [35.1, 35.1, 35.1, ... 35.1, 35.1, 35.1] type='781534 * float32[parameter...'>

In [3]:
# fname = '/project01/ndcms/hnelson2/ttbarEFT/nanoGen/TT01j2l_S1/nanoGen_101.root'
# fname = '/afs/crc.nd.edu/user/h/hnelson2/ttbarEFT/input_samples/nanoGen/djr_test_nanoGen.root'
# fname = '/cms/cephfs/data/store/user/hnelson2/tW_EFT/nanoGen/test_v2/NanoGen_tW_OGstpt/nanoGen_101.root'

# fname = '/cms/cephfs/data/store/user/hnelson2/tW_EFT/nanoGen/test_v4/NanoGen_tW_test/nanoGen_3.root'
# fname = '/cms/cephfs/data/store/user/hnelson2/tW_EFT/nanoGen/test_v5/NanoGen_tW_OGstpt/nanoGen_11.root'
# fname = '/cms/cephfs/data/store/user/hnelson2/mc/test/Jan24/NanoGen_TT01j2l_alt_stpt/nanoGen_1.root'
# fname = "/cms/cephfs/data/store/user/hnelson2/noEFT/nanoGen/tW/NanoGen_tW_noEFT/nanoGen_101.root"

fname = '/cms/cephfs/data/store/user/hnelson2/tW_EFT/nanoGen/fixed_rwgts/NanoGen_tW_OGstpt/nanoGen_365.root'

In [4]:
# Load in events from root file
events = NanoEventsFactory.from_root(
    fname,
    schemaclass=NanoAODSchema.v6,
    metadata={"dataset": "TLNuto2L2Nu"},
).events()

In [17]:
wc_lst = utils.get_list_of_wc_names(fname)
print(wc_lst)

['cHtbRe', 'ctGRe', 'ctGIm', 'cHQ3', 'cbWRe', 'cQl3', 'cleQt3Re', 'ctWRe', 'cleQt1Re']


In [5]:
print(len(events))

16000


In [6]:
print(events["genWeight"])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [7]:
eft_coeffs = events['EFTfitCoefficients']

1.0578204978628492e-12


In [16]:
print(len(eft_coeffs[0]))

55


In [9]:
print(eft_coeffs[1])

[0.00611, 4.86e-14, 1.72e-13, -0.000221, ... -8.25e-14, 2.95e-15, 1.9e-13, -9.43e-15]


In [10]:
print(eft_coeffs[2])

[0.00601, 8.22e-13, 2.89e-13, -0.000126, ... -1.36e-14, -6.28e-14, -3.11e-15]


In [11]:
# Initialize objects
genpart = events.GenPart
is_final_mask = genpart.hasFlags(["fromHardProcess","isLastCopy"])
ele  = genpart[is_final_mask & (abs(genpart.pdgId) == 11)]
mu   = genpart[is_final_mask & (abs(genpart.pdgId) == 13)]

In [12]:
######## Lep selection  ########
e_selec = ((ele.pt>20) & (abs(ele.eta)<2.5))
m_selec = ((mu.pt>20) & (abs(mu.eta)<2.5))
leps = ak.concatenate([ele[e_selec],mu[m_selec]],axis=1)
l0 = leps[ak.argmax(leps.pt, axis=-1, keepdims=True)]

In [13]:
leps = leps[ak.argsort(leps.pt, axis=-1, ascending=False)]

In [14]:
nleps = ak.num(leps)
# leps = leps[nleps==2]

In [18]:
proc_axis = hist.axis.StrCategory([], name="process", growth=True)
nleps_hist = HistEFT(proc_axis, hist.axis.Regular(bins=10, start=0, stop=10, name="nleps", label="nleps"), wc_names=wc_lst, label="Events")
event_weights = np.ones_like(events['event'])

In [19]:
print(type(nleps_hist))

<class 'topcoffea.modules.histEFT.HistEFT'>


In [22]:
print(len(eft_coeffs))

16000


In [24]:
nleps_hist.fill(nleps, process="tW", weight=event_weights, eft_coeff=eft_coeffs)

TypeError: fill() got multiple values for argument 'eft_coeff'

In [28]:
l0 = leps[ak.argmax(leps.pt, axis=-1, keepdims=True)]
l0 = leps[:,0]
l1 = leps[:,1]

In [29]:
dr = l0.delta_r(l1)

In [30]:
dr

<Array [2.81, 2, 2.85, ... 3.88, 3.36, 2.81] type='4754 * float32'>

In [ ]:
events.LHE.fields

In [ ]:
events.LHE.HTIncoming

In [ ]:
events.GenJet.fields

In [ ]:
events.LHEPart.fields

In [ ]:
events.GenPart.fields

In [ ]:
events.LHEWeight.fields

In [ ]:
events.LHEWeight.DJR10

In [ ]:
events.LHEWeight.DJR21

In [ ]:
events.LHEWeight.DJR32